In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
os.environ['NORMALIZATION_TYPE'] = 'zero_unit_variance'  

import sys
sys.path.append("..")
sys.path.append("../mango")
sys.path.append("../mango/design_baselines/diff")


import numpy as np
import torch
import pytorch_lightning as pl

import mango_jupyter_scripts
import design_bench
from helpers import seed_everything 

from SDE import DiffusionSOO

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
#!/usr/bin/env python3
"""
Main driver program for the mango as one of baseline methods for design-bench.

"""
import argparse
import logging
def build_args() -> argparse.Namespace:
    """
    Defines the experimental arguments for offline MBO baseline method
    evaluation.
    Input:
        None.
    Returns:
        A namespace containing the experimental argument values.
    """
    parser = argparse.ArgumentParser(description="Our method Experiments")

    parser.add_argument(
        "--task",
        type=str,
        default='Superconductor-RandomForest-v0', 
        choices=[task.task_name for task in design_bench.registry.all()],
        help="The name of the design-bench task for the experiment."
    )
    parser.add_argument(
        "--logging-dir",
        type=str,
        default=None,
        help="Logging directory to save optimization results to."
    )
    parser.add_argument(
        "--budget", type=int, default=2048, help="Query budget. Default 2048."
    )
    parser.add_argument(
        "--hidden-size",
        type=int,
        default=2048,
        help="Hidden size of the surrogate objective model. Default 2048."
    )
    parser.add_argument(
        "--seed", type=int, default=2024, help="Random seed. Default 42."
    )
    parser.add_argument(
        "--device", type=str, default="auto", help="Device. Default `auto`."
    )
    args = parser.parse_args([])
    return args

args = build_args()

In [ ]:
from mango_jupyter_scripts.training import train_surrogate, mango_train
# Training a surrogate forward model (It is merely to provide a forward model for comparison purposes only).
forwardmodel = train_surrogate(task_name=args.task, hidden_size=args.hidden_size)

Loading model from ../checkpoints/surrogate/Superconductor-RandomForest-v0-surrogate-100epoch.ckpt


In [ ]:
dataset_kwargs=dict(
    max_samples=10000,
    distribution=None,
    max_percentile=80,
    min_percentile=0) 

task = design_bench.make( args.task, dataset_kwargs=dataset_kwargs)
normalize_x = False
normalize_y = True
x_min, x_max, y_min, y_max = np.min(task.dataset.x, axis=0), np.max(task.dataset.x, axis=0), np.min(task.dataset.y, axis=0), np.max(task.dataset.y, axis=0) 
try:
    x1_range = task.dataset.x1_range
    x2_range = task.dataset.x2_range
    x_lbound = np.array([x1_range[0], x2_range[0]])
    x_ubound = np.array([x1_range[1], x2_range[1]])
except:
    x_lbound = task.oracle.internal_dataset.x.min(axis=0)
    x_ubound = task.oracle.internal_dataset.x.max(axis=0)
    y_lbound = task.oracle.internal_dataset.y.min()
    y_ubound = task.oracle.internal_dataset.y.max()
    
print("##################Before Normalization##################")
print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

if normalize_y:
    task.map_normalize_y()
    y_min = task.normalize_y(y_min)[0]
    y_max = task.normalize_y(y_max)[0]
    y_lbound = task.normalize_y(y_lbound)[0]
    y_ubound = task.normalize_y(y_ubound)[0]
if normalize_x:
    task.map_normalize_x()
    x_min = task.normalize_x(x_min)[0]
    x_max = task.normalize_x(x_max)[0]
    x_lbound = task.normalize_x(x_lbound)[0]
    x_ubound = task.normalize_x(x_ubound)[0]

if normalize_x or normalize_y:
    print("##################After Normalization##################")
    print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
    print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
    print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

##################Before Normalization##################
x_min: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
 x_max: [ 14.        0.        3.       35.      105.      120.       12.8
  66.        4.        0.        4.       12.       99.925   100.
  20.       15.        3.        0.        3.       24.        5.
  75.       76.4      34.9       6.       30.       35.38     45.
  98.       20.       41.       46.       18.       19.        3.
   0.        3.       11.        9.       96.71     99.078    99.992
   2.       64.       45.       50.99745   7.       99.995    31.5
  97.       75.       66.7       4.        0.        3.       24.
  98.        4.      185.        6.        0.        6.        6.
   4.        4.        2.67      5.        5.        5.       16.

In [9]:
import shutil
args = build_args()
 
num_epochs=  400

args.logging_dir = args.logging_dir or f"../logs/run_ours_{args.task}_seed{args.seed}_budget{args.budget}_hidden{args.hidden_size}_epoch{num_epochs}_betamax{0.05}_01normalization"
print(args.logging_dir )

logging_dir = args.logging_dir
version = 1
while os.path.exists(logging_dir):
    logging_dir = f"{args.logging_dir}_v{version}"
    version += 1
os.makedirs(logging_dir, exist_ok=True)

seed_everything(args.seed)
torch.set_default_dtype(torch.float32)
import warnings
warnings.filterwarnings("ignore", message="The dataloader, .* does not have many workers.*")

logging.basicConfig(
    level=logging.INFO,  # 设置最低日志级别
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # 设置日志格式
    handlers=[
        logging.FileHandler(f'{args.logging_dir}/running.log'),  # 将日志写入文件
        # logging.StreamHandler()  # 同时将日志输出到控制台
    ]
)

try:
    # # 尝试访问 inverse_model
    inverse_model

except NameError:
    inverse_model = None  # 如果未定义，设置为 None

 

# inverse_model = ours_method_train(
#     task=task,
#     task_name=args.task,
#     ckpt_dir=args.logging_dir,
#     hidden_size=args.hidden_size,
#     device=args.device,
#     seed=args.seed,
#     condition_training=False,
#     num_epochs=num_epochs,
#     dropout=0.0,
#     augment=True,
#     lr=5e-5,
#     batch_size=64,
#     debais=True,
# )



Global seed set to 2024


../logs/run_ours_Superconductor-RandomForest-v0_seed2024_budget2048_hidden2048_epoch400_betamax0.05_01normalization


In [ ]:

model_ckpt = f"/home/tzhouaq/ManGO_SOO/logs/run_ours_Superconductor-RandomForest-v0_seed2024_budget2048_hidden2048_epoch1000_betamax0.05_01normalization/ours-superconductor/superconductor-seed2024-hidden2048-score_matchingFalse.ckpt"
inverse_model = DiffusionSOO.load_from_checkpoint(
    model_ckpt,
    taskname=task.dataset_name,
    task=task,
    lr=1e-4,
    dropout_p=0.0,
    hidden_size=args.hidden_size,
    debais=True,
    augment=True,
    beta_min=0.01,
    beta_max=2,
    condition_training=False
)
# inverse_model.gen_sde.forwardmodel.load_state_dict(model.state_dict())
# inverse_model.forwardmodel.load_state_dict(model.state_dict())


uncondition_training T: {1.0}


In [ ]:
from mango_jupyter_scripts.eval_is import mango_eval
# This cell is to conduct unconditional generation

augment = True
condition_training = False
 
if augment:
    if not normalize_y:
        y_lbound = [y_lbound]
        y_ubound = [y_ubound]
    clip_min = torch.tensor(np.concatenate((x_lbound, y_lbound)))
    clip_max = torch.tensor(np.concatenate((x_ubound, y_ubound)))
else:
    clip_min = torch.tensor(x_min)
    clip_max = torch.tensor(x_max)
 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)

# y_given_list = np.linspace(0.5, 15, 200)
print('num_samples budget:', args.budget)
num_samples = args.budget
 
 
# y_given = torch.tensor([2])
res = mango_eval(
    task=task,
    task_name=args.task,
    forwardmodel=forwardmodel,
    inverse_model=inverse_model,
    device='auto',
    logging_dir=args.logging_dir,
    num_samples=num_samples,
    augment=augment,
    clip_dic=clip_dic,
    # gamma=0,
    condition_training=condition_training,
    num_steps=5,
    seed=args.seed,
 
)

res['x'].shape#, res['y'].shape, res['y_scores'].shape
 

{'simple_clip': True, 'clip_min': tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.9925]), 'clip_max': tensor([ 14.0000,   0.0000,   3.0000,  40.0000, 105.0000, 120.0000,  12.8000,
   

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


(2048, 86)

In [14]:

import numpy as np

# designs = np.load(os.path.join(args.logging_dir, "solution.npy"))
# preds = np.load(os.path.join(args.logging_dir, "predictions.npy"))
# scores = np.load(os.path.join(args.logging_dir, "scores.npy"))
y_unob_max = 185#task.oracle.internal_dataset.y.max()
y_unob_min = task.oracle.internal_dataset.y.min()

designs = res['x']
preds =   res['y_gen']
# preds = res['y']
scores = res['y_scores']

# designs = designs.reshape(1, -1, designs.shape[-1]) 
# preds = preds.reshape(1, -1, preds.shape[-1])
# scores = scores.reshape(1, -1, scores.shape[-1])
print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])

  
# task = design_bench.make(args.task)
if task.is_normalized_y:
    best_training_score = task.denormalize_y(np.max(task.dataset.y))#[0]
else:
    best_training_score = np.max(task.dataset.y)
print("Training dataset:\n The best score during training is: ",best_training_score )
oracle = np.max(scores)
print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
print(f' The corresponding design is: {designs[np.argmax(scores)]}')

k = 1
percentile = 100
one_shot = scores[np.argmax(preds)][0]
normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
 


k = 16
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

k = 128
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')
 

k = 256
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 77.62721252441406
 The corresponding design is: [7.8898743e-02 0.0000000e+00 0.0000000e+00 0.0000000e+00 3.8368714e-01
 0.0000000e+00 0.0000000e+00 4.3004041e+00 3.4725893e-01 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 6.1534494e-02 0.0000000e+00 0.0000000e+00 1.0715234e+00
 0.0000000e+00 0.0000000e+00 2.4722518e-01 9.6047562e-01 0.0000000e+00
 0.0000000e+00 0.0000000e+00 2.7910426e-01 1.3624820e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 7.3111212e-01 1.0030539e+00 1.5582004e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 4.9704975e-01
 3.45277

In [ ]:
augment = True
y_ubound = task.normalize_y(task.oracle.internal_dataset.y.max())[0]
if normalize_y:
    print('y_given before normalization:',  y_ubound[-1])
    y_given = torch.tensor(task.normalize_y(task.oracle.internal_dataset.y.max())[0])
    # y_given = torch.tensor(task.normalize_y(135)[0])
    # y_given = torch.tensor(y_ubound)
else:
    y_given = torch.tensor(task.oracle.internal_dataset.y.max())
 
print('y_given:', y_given)
if y_max.item() < y_given:
    y_max = y_given 
if  y_min.item() > y_given:
    y_min = y_given

if augment:
    clip_min = torch.tensor(np.concatenate((x_lbound, y_lbound)))
    clip_max = torch.tensor(np.concatenate((x_ubound, y_ubound)))
else:
    clip_min = torch.tensor(x_min)
    clip_max = torch.tensor(x_max)
 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)

 
# seed_list = [2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032]# , 2033, 2034
seed_list = [ 40, 41,42,   44, 45, 46, 47, 49] 
# seed_list = [ 39,40, 42, ] 

reported_results_1shot = []
reported_results_16shot = []
reported_results_128shot = []
reported_results_256shot = []

args.budget = 256
num_steps = 5
inference_scaling_bool = True
duplicated_time = 64
guidance_scals2= 10
description = 'nsmc'

# y_given = torch.tensor([2])
for seed in seed_list:
    print(f'***********************seed{seed}*************************')
    print(f"Running ours method for {args.task} with seed {seed}")
    
    res = mango_eval(
        task,
        task_name=args.task,
        forwardmodel=forwardmodel,
        inverse_model=inverse_model,
        logging_dir=args.logging_dir,
        hidden_size=args.hidden_size,
        num_samples=args.budget,
        device=args.device,
        seed=seed,
        clip_dic=clip_dic,
        augment=augment,
        gen_condition=y_given,
        guidance=True,
        num_steps=num_steps,
        inference_scaling_bool=inference_scaling_bool,
        duplicated_time=duplicated_time,
        guidance_scals2=guidance_scals2,
        description=description
    )
 

    # designs = np.load(os.path.join(args.logging_dir, "solution.npy"))
    # preds = np.load(os.path.join(args.logging_dir, "predictions.npy"))
    # scores = np.load(os.path.join(args.logging_dir, "scores.npy"))
    y_unob_max = task.oracle.internal_dataset.y.max()
    y_unob_min = task.oracle.internal_dataset.y.min()

    designs = res['x']
    preds =   res['y_gen']
    # preds = res['y']
    scores = res['y_scores']

    # designs = designs.reshape(1, -1, designs.shape[-1]) 
    # preds = preds.reshape(1, -1, preds.shape[-1])
    # scores = scores.reshape(1, -1, scores.shape[-1])
    print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
    print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])

    
    # task = design_bench.make(args.task)
    if task.is_normalized_y:
        best_training_score = task.denormalize_y(np.max(task.dataset.y))#[0]
    else:
        best_training_score = np.max(task.dataset.y)
    print("Training dataset:\n The best score during training is: ",best_training_score )
    oracle = np.max(scores)
    print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
    print(f' The corresponding design is: {designs[np.argmax(scores)]}')

    k = 1
    percentile = 100
    one_shot = scores[np.argmax(preds)][0]
    normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
    reported_results_1shot.append(normalize_one_shot)

    k = 4
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_16shot.append(normalize_few_shot)

    k = 16
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_16shot.append(normalize_few_shot)

    k = 128
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_128shot.append(normalize_few_shot)

    k = 256
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_256shot.append(normalize_few_shot)
    
print(f'***********************OVERALL RESULTS*************************')
print(f"reported_results_1shot: {np.mean(reported_results_1shot):.4f} ± {np.std(reported_results_1shot):.4f}")
print(f"reported_results_16shot: {np.mean(reported_results_16shot):.4f} ± {np.std(reported_results_16shot):.4f}")
print(f"reported_results_128shot: {np.mean(reported_results_128shot):.4f} ± {np.std(reported_results_128shot):.4f}")
print(f"reported_results_256shot: {np.mean(reported_results_256shot):.4f} ± {np.std(reported_results_256shot):.4f}")

y_given before normalization: 8.099974
y_given: tensor([8.1000])
{'simple_clip': True, 'clip_min': tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.9925]), 'clip_max': tensor([ 14.0000

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 99.92436218261719
 The corresponding design is: [0.37633988 0.         0.         0.05722916 0.19405523 0.
 0.         0.8623703  0.4368837  0.         0.         0.20542341
 0.05224789 0.12506291 0.         0.         0.3384012  0.
 0.0043231  2.9428904  0.         0.         0.         0.
 0.         1.599224   0.         0.5322782  2.401057   1.5265698
 0.         0.         0.         0.         0.17267579 0.
 0.         1.6748306  0.59188586 0.         0.         0.
 0.         0.         0.03631452 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         1.8038626  0.5724127  0.         0.         0.
 0.         0.29033393 0.05274597 0.      

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 110.02481079101562
 The corresponding design is: [0.         0.         0.         0.         0.         0.
 0.         3.7746918  0.         0.         0.         0.18340367
 0.         0.         0.         0.         0.         0.
 0.         2.0833285  0.         0.         1.1830742  0.
 0.         0.         0.         0.12106261 1.909199   0.6387709
 1.2493854  0.         0.24562421 0.28143546 0.         0.
 0.97231114 1.8027188  0.         0.         0.         0.
 1.2631242  0.         0.         0.         0.         0.
 0.         0.403842   0.         0.21640389 0.         0.
 0.12876141 2.218946   0.         0.         0.         0.24383205
 0.         0.22774538 0.        

...

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 100.105712890625
 The corresponding design is: [0.         0.         0.         0.         0.         0.
 0.         3.1161761  0.40418893 0.         0.         0.11869179
 0.         0.         0.         0.         0.3972741  0.
 0.         1.1909734  0.         0.3433685  0.         0.
 0.05113297 0.         0.         0.         2.898756   0.
 0.2692611  0.         0.1380074  0.32173893 0.         0.
 1.206316   1.9627979  0.         0.         0.         0.
 0.03992134 0.         0.17004834 0.         0.9835312  0.
 0.         0.         0.         0.         0.         0.
 0.3041583  2.146224   0.61746216 0.04800761 0.30871964 0.
 0.         0.32068986 0.         0.64961886 0.   

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  86 ; the dimension of the designs is:  86
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  [[74.]]
Oracle results:
 On the given design: the Oracle is: 105.29118347167969
 The corresponding design is: [0.         0.         0.         0.         0.         0.
 0.         0.27529955 0.         0.         0.         0.
 0.         0.         0.13457246 0.07826062 0.30064255 0.
 0.         2.715733   0.7644671  0.         0.47823608 0.
 0.         0.         0.05077398 0.34969807 1.9283109  0.83442223
 0.         0.         0.         0.01381901 0.30734342 0.
 0.61747396 2.833296   0.         0.         0.         0.31192836
 0.12743708 0.02489743 0.         0.         0.         0.
 0.         0.8080817  0.         0.         0.11807911 0.
 0.         2.7580216  0.63642645 0.         0.29158497 0.
 0.         0.27574983 0.         0.    

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
